In [ ]:
# Set number of cores for parallel computation
# For just reading output files and creating summary, use 1 kernel
KERNEL = 38


import os
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
import pandas as pd
import sys,os
import random
import copy

import matplotlib.pyplot as plt
import seaborn as sns

from utils.eval import find_best_matches, generate_exprs

from methods import NMF, PCA, sparse_PCA, moCluster, MOFA2

from methods.utils import interpret_results, resultsHandler

from pathlib import Path
import multiprocessing as mp

from utils.eval import find_best_matches, make_known_groups, find_best_matching_biclusters


def make_ref_groups(subtypes, annotation,exprs):
    # prepared a dict of subtype classifications {"class1":{"subt1":[],"subt2":[]},"class2":{"subtA":[],"subtB":[]}}
    all_samples = set(exprs.columns.values)
    pam50 = make_known_groups(subtypes, exprs,target_col = "PAM50",verbose=False)
    lum = {}
    lum["Luminal"] = pam50["LumA"].union(pam50["LumB"])
    scmod2 = make_known_groups(subtypes, exprs,target_col = 'SCMOD2',verbose=False)
    claudin = {} 
    claudin["Claudin-low"] = set(subtypes.loc[subtypes['claudin_low']==1,:].index.values).intersection(all_samples)
    
    ihc = {}
    for x in ["IHC_HER2","IHC_ER","IHC_PR"]:
        ihc[x] = set(annotation.loc[annotation[x]=="Positive",:].index.values)
    ihc["IHC_TNBC"] = set(annotation.loc[annotation["IHC_TNBC"]==1,:].index.values)
    
    known_groups = {"PAM50":pam50,"Luminal":lum,"Claudin-low":claudin,"SCMOD2":scmod2,"IHC":ihc}
    
    weights = {}
    N = 0
    for classification in known_groups.keys():
        for group in known_groups[classification].keys():
            n = len(known_groups[classification][group])
            weights[group] = n
            N+=n
    for group in weights.keys():
        weights[group] = weights[group]/N
    
    return known_groups, weights

def calculate_perfromance(results, known_groups, weights, all_samples):
    # finds best matches for each subtype, calcuates J per subtype and overall performance
    N = len(all_samples)
    best_matches = []
    
    for classification in known_groups.keys():
        bm = find_best_matches(results,known_groups[classification],all_samples,FDR=0.05,verbose = False)
        best_matches.append(bm)
            
    best_matches = pd.concat(best_matches, axis=0)
    best_matches = best_matches["J"].to_dict()
    overall_performance = 0
    for group in best_matches.keys():
        overall_performance += best_matches[group]*weights[group]
    best_matches["overall_performance"] = overall_performance
    return best_matches

def compare_gene_clusters(tcga_result,metabric_result, N):
    # N - total number of genes
    # finds best matched TCGA -> METABRIC and METABRIC -> TCGA
    # calculates % of matched clusterst, number of genes in matched cluster, 
    # and the average J index for best matches 
    bm = find_best_matching_biclusters(tcga_result,metabric_result, N)
    bm = bm.dropna()
    bm2 = find_best_matching_biclusters(metabric_result, tcga_result, N)
    bm2 = bm2.dropna()
    
    bm = bm.loc[bm["n_shared"]>1,:].sort_values(by="n_shared",ascending = False)
    bm2 = bm2.loc[bm2["n_shared"]>1,:].sort_values(by="n_shared",ascending = False)
    
    
    clust_similarity = {}
    # number of biclusters 
    clust_similarity["n_1"] = tcga_result.shape[0]
    clust_similarity["n_2"] = metabric_result.shape[0]
    #print("% matched biclusters:",bm.shape[0]/tcga_result.shape[0],bm2.shape[0]/metabric_result.shape[0])
    clust_similarity["percent_matched_1"] = bm.shape[0]/tcga_result.shape[0]
    clust_similarity["percent_matched_2"] = bm2.shape[0]/metabric_result.shape[0]
    #print("n matched genes:",bm.loc[:,"n_shared"].sum(),bm2.loc[:,"n_shared"].sum())
    clust_similarity["n_shared_genes_1"] = bm.loc[:,"n_shared"].sum()
    clust_similarity["n_shared_genes_2"] = bm2.loc[:,"n_shared"].sum()
    #print("avg. J:",bm.loc[:,"J"].mean(),bm2.loc[:,"J"].mean())
    clust_similarity["avg_bm_J_1"] = bm.loc[:,"J"].mean()
    clust_similarity["avg_bm_J_2"] = bm2.loc[:,"J"].mean()
    
    
    return clust_similarity, bm, bm2




file_metabric_annotation = '/home/hartung/data/preprocessed_v6/METABRIC_1904.annotation_v6.tsv'
file_metabric_expression = '/home/hartung/data/preprocessed_v6/METABRIC_1904_17Kgenes.log2_exprs_z_v6.tsv'
file_metabric_subtypes = '/home/hartung/data/preprocessed_v6/METABRIC_1904_17Kgenes.subtypes_and_signatures_v6.tsv'
file_tcga_annotation = '/home/hartung/data/preprocessed_v6/TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv'
file_tcga_expression = '/home/hartung/data/preprocessed_v6/TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv'
file_tcga_subtypes = '/home/hartung/data/preprocessed_v6/TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv'
file_gene_mapping = '/home/hartung/data/preprocessed_v6/gene_id_mapping.tsv'

out_dir= '/home/hartung/data/preprocessed_v6/results'

basename_t = "TCGA"
basename_m = "METABRIC" 


m_subtypes = pd.read_csv(file_metabric_subtypes,sep = "\t",index_col=0)
m_annotation = pd.read_csv(file_metabric_annotation,sep = "\t",index_col=0)

t_subtypes = pd.read_csv(file_tcga_subtypes,sep = "\t",index_col=0)
t_annotation = pd.read_csv(file_tcga_annotation,sep = "\t",index_col=0)


exprs_t= pd.read_csv(file_tcga_expression,sep = "\t",index_col=0)
exprs_t[exprs_t>3] = 3
exprs_t[exprs_t<-3] = -3

exprs_m= pd.read_csv(file_metabric_expression,sep = "\t",index_col=0)
exprs_m[exprs_m>3] = 3
exprs_m[exprs_m<-3] = -3

known_groups_t, weights_t = make_ref_groups(t_subtypes, t_annotation,exprs_t)
known_groups_m, weights_m = make_ref_groups(m_subtypes, m_annotation,exprs_m)


METHODS = [moCluster, MOFA2] # [NMF, sparse_PCA, moCluster, MOFA2]
for METHOD in METHODS:
    method_name = METHOD.__name__.split('.')[-1]
    
    #### Preparation
    # METABRIC
    file_path_m = file_metabric_expression
    output_path_m = os.path.join(out_dir, basename_m, method_name)
    ground_truth_file_m = file_metabric_annotation
    combinations_m = METHOD.generate_arg_list(file_path_m, output_path_m, ground_truth_file_m)
    # TCGA
    file_path_t = file_tcga_expression
    output_path_t = os.path.join(out_dir, basename_t, method_name)
    ground_truth_file_t = file_tcga_annotation
    combinations_t = METHOD.generate_arg_list(file_path_t, output_path_t, ground_truth_file_t)

    
    #### Compute in parallel
    # Option to compute the results in parallel, methods will store results
    # Follow up with executing the 'Run' below to read existing results and evaluate
    if KERNEL > 1:
        with mp.Pool(KERNEL) as pool:
            pool.map(METHOD.run_real, combinations_m + combinations_t)

            
    #### Run
    # Methods will compute results or read existing results
    # sanity check
    assert len(combinations_m) == len(combinations_t)
    subt_t = []
    subt_m = []
    clustering_similarities = []
    for comb_m, comb_t in zip(combinations_m, combinations_t):
        result_m, runtime_m = METHOD.run_real(comb_m)
        result_t, runtime_t = METHOD.run_real(comb_t)
        
        try:
            performance_m = calculate_perfromance(result_m, known_groups_m,
                                                  weights_m, set(exprs_m.columns.values))
            performance_m.update(comb_m)
            performance_m['time'] = runtime_m
        except ZeroDivisionError:
            performance_m = {}
        subt_m.append(performance_m)

        try:
            performance_t = calculate_perfromance(result_t, known_groups_t,
                                                  weights_t, set(exprs_t.columns.values))
            performance_t.update(comb_t)
            performance_t['time'] = runtime_t
        except ZeroDivisionError:
            performance_t = {}
        subt_t.append(performance_t)

        
    # save results
    pd.DataFrame.from_records(subt_m).to_csv(os.path.join(out_dir, basename_m, method_name, 'best_cluster_matches.csv'))    
    pd.DataFrame.from_records(subt_t).to_csv(os.path.join(out_dir, basename_t, method_name, 'best_cluster_matches.csv'))


Returning existing results:Returning existing results:Running /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=11/n_cluster=4/random_state=1...  /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=1/n_cluster=4/random_state=1Returning existing results:
/home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=6/n_cluster=4/random_state=1
Running /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=11/n_cluster=5/random_state=1...Running /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=16/n_cluster=4/random_state=1...Running /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=16/n_cluster=7/random_state=1...Returning existing results:Returning existing results:Running /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=16/n_cluster=5/random_state=1...Returning existing results:Running /home/hartung/data/preprocessed_v6/results/ME

Returning existing results:Returning existing results: Returning existing results:  /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=8/n_cluster=5/random_state=1Returning existing results: Returning existing results: Returning existing results:/home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=10/n_cluster=8/random_state=1
  /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=3/n_cluster=9/random_state=1Returning existing results: 
/home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=8/n_cluster=9/random_state=1 /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=9/n_cluster=6/random_state=1 /home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=14/n_cluster=10/random_state=1Returning existing results:
/home/hartung/data/preprocessed_v6/results/METABRIC/moCluster/n_dimensions=8/n_cluster=10/random_state=1/home/hartung/data/preprocessed_v6/results/MET

Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
L

      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]

Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table
Loading required package: data.table


      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
      [,1]
[1,] 17158
[2,]  1904
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating component 3 ...
calculating c

Loading required package: data.table
Loading required package: data.table
Loading required package: data.table


In [ ]:
import pandas as pd


df = pd.read_csv('/home/hartung/data/preprocessed_v6/results/METABRIC/MOFA2/best_cluster_matches.csv')